In [1]:
### Chapter 7: compulsory: 3 and 5 and select one of these two sets: {1} or {2}
from __future__ import division
from __future__ import print_function
import pandas as pd
%pylab inline
import time
import warnings
import os
from collections import Counter
from itertools import chain
import json
import seaborn as sns
import ast
with warnings.catch_warnings():
    warnings.filterwarnings("ignore",category=DeprecationWarning)
import sklearn.preprocessing

print("Imports done ...✈✈✈✈✈✈✈✈ You are free to go.")

import six
from sklearn.datasets import load_breast_cancer
from parser import generate_labels,get_sensor_values,get_labels,generate_csv

Populating the interactive namespace from numpy and matplotlib
Imports done ...✈✈✈✈✈✈✈✈ You are free to go.


In [2]:
dataset_folder = './ml4qsbeasts/'
fname = 'new_log'
dataset_path = os.path.join(dataset_folder,fname)
print(dataset_path)
print(os.getcwd())
from itertools import izip

def pairwise(iterable):
    "s -> (s0, s1), (s2, s3), (s4, s5), ..."
    a = iter(iterable)
    return izip(a, a)

./ml4qsbeasts/new_log
/home/cts/Desktop/ML4QS/Assignments/assignments_code_git/assignment2


In [3]:
sensor_dict = get_sensor_values(dataset_path,device='tommie_device')
generate_csv(sensor_dict['bmi160 accelerometer'],header=['x,y,z','timestamp'],fname='tmp_acce.csv')
generate_labels()

{'standing': [6, 11], 'walking': [9, 12], 'sitting': [10, 13], 'stairsdown': [4, 7], 'stairsup': [5, 8], 'ondesk': [14]}


[['interval_label',
  'tommie smartphone',
  'stairsdown',
  '1498010210795',
  Timestamp('2017-06-21 01:56:50.795000'),
  '1498010250904',
  Timestamp('2017-06-21 01:57:30.904000')],
 ['interval_label',
  'tommie smartphone',
  'stairsup',
  '1498010251062',
  Timestamp('2017-06-21 01:57:31.062000'),
  '1498010297381',
  Timestamp('2017-06-21 01:58:17.381000')],
 ['interval_label',
  'tommie smartphone',
  'standing',
  '1498010297414',
  Timestamp('2017-06-21 01:58:17.414000'),
  '1498010337044',
  Timestamp('2017-06-21 01:58:57.044000')],
 ['interval_label',
  'tommie smartphone',
  'stairsdown',
  '1498010337074',
  Timestamp('2017-06-21 01:58:57.074000'),
  '1498010376387',
  Timestamp('2017-06-21 01:59:36.387000')],
 ['interval_label',
  'tommie smartphone',
  'stairsup',
  '1498010376422',
  Timestamp('2017-06-21 01:59:36.422000'),
  '1498010422415',
  Timestamp('2017-06-21 02:00:22.415000')],
 ['interval_label',
  'tommie smartphone',
  'walking',
  '1498010422433',
  Timestamp

# Chapter 7.3
In our example, we use the same methods, but on a classification task with the same labels as the crowdsignal dataset.

In [ ]:
print(len(dd))